In [7]:
%pip install -e ..

Obtaining file:///Users/sethvanderbijl/Coding%20Projects/DM_project_2
  Attempting uninstall: DM-project-2
    Found existing installation: DM-project-2 1.0
    Uninstalling DM-project-2-1.0:
      Successfully uninstalled DM-project-2-1.0
  Running setup.py develop for DM-project-2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from json import load
from data.load import load_data
import os
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, StackingClassifier, StackingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, HuberRegressor, LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import std
import time
import matplotlib.pyplot as plt
import copy
from helpers.ranking import to_ranking


/usr/local/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [44]:

# Load data
df = load_data(add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)

DF with these specific params is cached, returning cached version.


In [46]:
# Split into target and predictors
y = df['booking_bool']
X = df.drop(['booking_bool','click_bool', 'position', 'gross_bookings_usd', 'date_time'], axis=1)
X = X.fillna(X.mean())
del df

In [5]:
# Set up stakcing regressor
# rf = RandomForestRegressor()
# rf= StackingRegressor([('rf',RandomForestRegressor()), ('br',BayesianRidge()), ('ab',AdaBoostRegressor())], verbose=3, n_jobs=7)
rf=RandomForestRegressor(verbose=3)
rf.fit(X, y)

RandomForestRegressor()

In [14]:
import joblib
_ = joblib.dump(rf, 'model.pkl')

In [49]:
# Load our own test data
test = load_data(mode='our_test', add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)


No cache for this specific request, start loading base df from disk.


100%|██████████| 6/6 [00:00<00:00, 40.31it/s]

generating dayparts
1 / 6 Generating datetimes for early_morning
2 / 6 Generating datetimes for morning
3 / 6 Generating datetimes for noon
4 / 6 Generating datetimes for eve
5 / 6 Generating datetimes for night
6 / 6 Generating datetimes for late_night



100%|██████████| 2/2 [00:00<00:00, 38.71it/s]


generating seasons
generating same id features
Feature engineering columns with operations
saving result of specific arguments to cache


100%|██████████| 500/500 [00:08<00:00, 61.13it/s]


In [57]:
# Drop irrelevant columns from our own test data
X_test = test.drop(['date_time', 'gross_bookings_usd', 'booking_bool', 'click_bool', 'position'], axis=1)
X_test = X_test.fillna(X.mean()) #Mean of x or mean of x_test?

In [58]:
# Check if there is a difference in columns (there shouldn't be)
list(set(X_test.columns) - set(X.columns))

[]

In [68]:
# Predicting on our own test data
from helpers.ndcg import ndcg_eval

preds = rf.predict(X_test)

ranking = to_ranking(X_test, preds)

score = ndcg_eval(ranking, test, show_difference=True)

score


TypeError: ndcg_eval() got an unexpected keyword argument 'show_difference'

In [64]:
# So now let's predict for the kaggle_competition
kaggle_test = load_data(mode='kaggle_test', add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)

No cache for this specific request, start loading base df from disk.


  0%|          | 0/6 [00:00<?, ?it/s]

generating dayparts
1 / 6 Generating datetimes for early_morning


 17%|█▋        | 1/6 [00:24<02:00, 24.02s/it]

2 / 6 Generating datetimes for morning


...

100%|██████████| 2/2 [00:40<00:00, 20.48s/it]


Feature engineering columns with operations


  0%|          | 0/500 [00:00<?, ?it/s]

saving result of specific arguments to cache


100%|██████████| 500/500 [01:11<00:00,  7.04it/s]


In [77]:
# Drop irrelevant columns from our own test data
X_test_kaggle = kaggle_test.drop('date_time', axis=1)
X_test_kaggle = X_test_kaggle.fillna(X.mean()) #Mean of x or mean of x_test?

# Again double check whether columns are equal
list(set(X_test_kaggle.columns) - set(X.columns))

[]

In [78]:
# The moment of truth :O
preds = rf.predict(X_test_kaggle)

ranking = to_ranking(X_test_kaggle, preds)

ranking

,srch_id,prop_id
25,1,128085
6,1,34263
8,1,50162
23,1,99484
5,1,28181
...,...,...
4959177,332787,29018
4959178,332787,32019
4959181,332787,94437
4959182,332787,99509


In [80]:
ranking.to_csv('kaggle_ranking_seth.txt', index=False)

In [79]:
# Also look at feature importances, both fr debugging and informing group mates
l = zip(X.columns, rf.feature_importances_)
df = pd.DataFrame(l, columns=['f', 'i']).sort_values('i', ascending=False)
print(df)
df.to_csv('temp feature importances')

                                 f         i
60  same_val_srch_id_price_usd_avg  0.075240
13                       price_usd  0.073632
0                          srch_id  0.071089
23       orig_destination_distance  0.070275
11            prop_location_score2  0.063259
..                             ...       ...
35                       comp4_inv  0.002395
41                       comp6_inv  0.002080
27         comp1_rate_percent_diff  0.001900
25                      comp1_rate  0.001200
26                       comp1_inv  0.000753

[62 rows x 2 columns]
